In [24]:
import numpy as np
import tensorflow as tf

# TensorFlow Core tutorial

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [3]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [4]:
node3 = tf.add(node1, node2)
print('node3: ',node3)
print('sess.run(node3):', sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


In [5]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b

In [7]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


In [8]:
add_and_tripple = adder_node * 3.
sess.run(add_and_tripple, {a: 3, b: 4.5})

22.5

In [9]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

In [11]:
init = tf.global_variables_initializer()
sess.run(init)

In [12]:
sess.run(linear_model, {x: [1, 2, 3, 4]})

array([ 0.        ,  0.30000001,  0.60000002,  0.90000004], dtype=float32)

In [14]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
sess.run(loss, {x: [1,2,3,4], y: [0,-1,-2,-3]})

23.66

In [15]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1,2,3,4], y: [0,-1,-2,-3]}))

0.0


In [16]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [21]:
sess.run(init)
for i in range(1000):
    sess.run(train, {x: [1,2,3,4], y: [0,-1,-2,-3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


In [23]:
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: [1,2,3,4], y: [0,-1,-2,-3]})
print('W: %s b: %s loss: %s'%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


# tf.estimator

In [26]:
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

In [27]:
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpj79hely2', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [30]:
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

In [31]:
estimator.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpj79hely2/model.ckpt.
INFO:tensorflow:loss = 5.0, step = 1
INFO:tensorflow:global_step/sec: 312.196
INFO:tensorflow:loss = 0.0495568, step = 101 (0.322 sec)
INFO:tensorflow:global_step/sec: 329.771
INFO:tensorflow:loss = 0.00895487, step = 201 (0.304 sec)
INFO:tensorflow:global_step/sec: 341.614
INFO:tensorflow:loss = 0.00300643, step = 301 (0.293 sec)
INFO:tensorflow:global_step/sec: 342.149
INFO:tensorflow:loss = 0.00083453, step = 401 (0.292 sec)
INFO:tensorflow:global_step/sec: 330.402
INFO:tensorflow:loss = 4.47148e-05, step = 501 (0.303 sec)
INFO:tensorflow:global_step/sec: 309.655
INFO:tensorflow:loss = 5.57788e-06, step = 601 (0.323 sec)
INFO:tensorflow:global_step/sec: 323.043
INFO:tensorflow:loss = 3.00294e-07, step = 701 (0.310 sec)
INFO:tensorflow:global_step/sec: 340.319
INFO:tensorflow:loss = 6.57335e-08, step = 801 (0.293 sec)
INFO:tensorflow:global_step/sec: 356.461
INFO:ten

In [32]:
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Starting evaluation at 2017-12-23-14:47:56
INFO:tensorflow:Restoring parameters from /tmp/tmpj79hely2/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-23-14:47:59
INFO:tensorflow:Saving dict for global step 1000: average_loss = 3.39564e-10, global_step = 1000, loss = 1.35825e-09
INFO:tensorflow:Starting evaluation at 2017-12-23-14:47:59
INFO:tensorflow:Restoring parameters from /tmp/tmpj79hely2/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-12-23-14:48:02
INFO:tensorflow:Saving dict for global step 1000: average_loss = 0.0025265, global_step = 1000, loss = 0.010106
train metrics: {'average_loss': 3.3956352e-10, 'loss': 1.3582541e-09, 'global_step': 1000}
eval metrics: {'average_loss': 0.0025264996, 'loss': 0.010105998, 'global_step': 1000}
